# SAAF Notebook

This Jupyter Notebook provides an interactive platform for FaaS development. 

In [ ]:
#
# Welcome to the SAAF Jupyter notebook! This default notebook provides comments to guide you through
# all of the main features. If you run into errors or problems please make sure you have the AWS CLI
# properly configure so that you can deploy function with it, have Docker installed and running, gave 
# execute permission to everything in the /jupyter_workspace and /test directory, and finally
# installed all the dependencies. You can use quickInstall.sh in the root folder to walk you through the
# setup process and install dependencies.
#
# This first cell is just imports needed to setup the magic that goes on behind the scenes. 
# Run it and it should return nothing.
#
# Function available in jupyter_workspace/platforms/jupyter/interactive_helpers.py
import os
import sys
sys.path.append(os.path.realpath('..'))
from platforms.jupyter.interactive_helpers import *

# Configure your function details here. Currently the only thing you need is a lambda ARN to assign to functions.
config = {
    "lambdaRoleARN": "FILL THIS IN"
}

# If you want to disable automatic deployment across the entire notebook change this.
setDeploy(True)

# Functions

Any function with the @cloud_function decorator will be uploaded to the cloud. Define platforms and memory settings in the decorator. 
Functions are tested locally and must run sucessfully before being deployed.

In [ ]:
#
# Here is your first cloud function! Creating cloud functions is as simple as writing python functions with (request, context)
# arguments and adding the @cloud_function decorator. Define the platform you would like to deploy to, the memory setting, and
# pass your context object defined earlier. Other arguments like references, requirements, and containerize can be used. 
# They will be shown later.
#
# Cloud functions defined in this notebook do have a few limitations. The main one is that nothing outside the function
# is deployed to the cloud. That is why imports are inside the function, which is a little weird and can have an
# effect on what you can import. But for most things this is fine. 
#
# Alongside deploying your function code, you can deploy files alongside this function by adding them to the
# src/includes_{function name} folder (This function will use src/includes_hello_world). 
# This folder will be automatically created when the function is ran. You can include basically anything, files, scripts,
# python libraries, whatever you need.
#
# If everything is setup correct, all you need to do is run this code block and you'll get a hello_world function
# on AWS Lambda! If not all dependencies are installed you can use ./quickInstall.sh to download dependencies.
#

@cloud_function(platforms=[Platform.AWS], memory=256, config=config)
def hello_world(request, context): 
    from Inspector import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
    inspector.addAttribute("message", "Hello from the cloud " + str(request["name"]) + "!") 
    
    inspector.inspectAllDeltas()
    return inspector.finish()

result = test(function=hello_world, payload={"name": "Bob"}, config=config)

In [ ]:
#
# What if we want one cloud function to call another function? The cloud function decorator can do that for you automatically!
# Simply add any cloud functions that this function calls to the references list. 
#
# This function isn't cheating and just deploying both hello_world and jello_world together, both are deployed 
# as seperate functions and making requests to the other. This example isn't practical but all features
# of python, such as multithreading, can be used to make multiple requests to functions in parallel.
#
# After running, see src/handler_jello_world.py for the automatically generated source code.
#

@cloud_function(platforms=[Platform.AWS], 
               memory=256, 
               config=config, 
               references=[hello_world])

def jello_world(request, context): 
    from Inspector import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
    cloud_request = hello_world(request, None)
    hello_message = cloud_request['message']
    jello_message = hello_message.replace("Hello", "Jello")
    inspector.addAttribute("message", jello_message)
    
    inspector.inspectAllDeltas()
    return inspector.finish()

result = test(function=jello_world, payload={"name": "Bob"}, config=config)

In [ ]:
#
# This function here requires the igraph dependency, you can see it defined in the requirements argument of the decorator.
# Alongside that, this function will be deployed as a container isn't of zip function. Containers are built,
# submitted to ECR, and deployed to AWS Lambda. For all function builds, you can see the generated files in 
# the /deploy directory. The complete build for this function will be in /deploy/graph_rank_container_aws_build where
# you will be able to see all the python files, dependencies, and Dockerfile. 
#
# This folder will be destroyed and recreated every time a function is deployed so it is not recommended to manually edit. 
# You can manually edit it and redeploy if it is really necessary.
#
@cloud_function(platforms=[Platform.AWS], memory=512, config=config, requirements="python-igraph", containerize=True)
def page_rank_container(request, context):
    from Inspector import Inspector 
    import datetime
    import igraph
    import time
    
    inspector = Inspector()
    inspector.inspectAll()
    
    size = request.get('size')
    loops = request.get('loops')

    for x in range(loops):
        graph = igraph.Graph.Tree(size, 10)
        result = graph.pagerank()

    inspector.inspectAllDeltas()
    return inspector.finish()

result = test(function=page_rank_container, payload={"size": 100000, "loops": 50}, config=config)

# Execute Experiments

Use FaaS Runner to execute complex FaaS Experiments.

In [ ]:
#
# Now, what's cooler than running a function on the cloud once? Running it multiple times! The run_experiment
# function allows you to create complex FaaS experiments. This function uses our FaaS Runner application
# to execute functions behind the scenes. It's primary purpose is to run multiple function requests across many threads. 
# You define payloads in the payloads list, choose your memory setting (it will switch settings automatically)
# and define how many runs you want to do, across how many threads, and how many times you want to repeat the test
# with iterations. These are the most important parameters, but there are many more defined in the link below. 
#
# After an experiment runs the results are converted into a pandas dataframe that you can continue using in this notebook. 
# For example you can use matplotlib to generate graphs (see below), or do any other form of data processing. 
#
# This function provides a lot of utility and functionality but does have some minor limitations compared to the
# actual FaaS Runner application.
# 1. You only can define a single memory setting per experiment. Results will be lost if multiple are used.
# 2. Categorization functionality is not included. It's functionality can be easily replicated.
# 3. Experiment results are saved to /test/history/interactiveExperiment and are DELETED whenever run_experiment is called.
#     If you would like to save experiment results permenantly back up that folder. 
#
# Below are two different experiments for our functions. Execute them and generate graphs using the code cell below.
# You now have experienced all the functionality of the SAAF Jupyter Workspace! Happy FaaS developing!
#

# Define experiment parameters. For more detail see: https://github.com/wlloyduw/SAAF/tree/master/test
hello_experiment = {
  "payloads": [{"name": "Bob"}],
  "memorySettings": [256],
  "runs": 25,
  "threads": 5,
  "iterations": 4,
  "warmupBuffer": 0,
  "sleepTime": 0,
  "randomSeed": 42,
  "showAsList": [],
  "showAsSum": ["newcontainer"],
  "ignoreFromAll": ["zAll", "version", "linuxVersion", "hostname"],
  "invalidators": {},
  "removeDuplicateContainers": False,
  "overlapFilter": "functionName",
  "openCSV": False
}

# Execute experiment
hello_world_results = run_experiment(function=hello_world, platform=Platform.AWS, experiment=hello_experiment, config=config)

In [ ]:
# Define experiment parameters. For more detail see: https://github.com/wlloyduw/SAAF/tree/master/test
page_rank_experiment = {
  "payloads": [{"size": 50000, "loops": 5},
                {"size": 100000, "loops": 5},
                {"size": 150000, "loops": 5}],
  "memorySettings": [512],
  "runs": 100,
  "threads": 100,
  "iterations": 2,
  "warmupBuffer": 1,
  "sleepTime": 0,
  "randomSeed": 42,
  "showAsList": [],
  "showAsSum": ["newcontainer"],
  "ignoreFromAll": ["zAll", "version", "linuxVersion", "hostname"],
  "invalidators": {},
  "removeDuplicateContainers": False,
  "overlapFilter": "functionName",
  "openCSV": True
}

# Execute experiment
page_rank_results = run_experiment(function=page_rank_container, platform=Platform.AWS, experiment=page_rank_experiment, config=config)

# Process Results

FaaS Runner experiment results are parsed into a Pandas dataframe. This flexibility allows the ability to perform any kind of data processing that you would like.

In [ ]:
# Import matplotlib and setup display.
import matplotlib.pyplot as plt
%matplotlib inline

# Histogram of runtime
plt.hist(hello_world_results['userRuntime'], 10)

In [ ]:
# Import matplotlib and setup display.
import matplotlib.pyplot as plt
%matplotlib inline

# Histogram of runtime
plt.hist(page_rank_results['userRuntime'], 10)